## Methos: 基于蛋白结构的反应预测
> 2024-12-20

### 1. 导入必要的包

In [1]:
import pandas as pd
import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../')
from config import conf as cfg
from tqdm import tqdm
from tools import btools
from tools import filetool 
import rxnrecer as production
from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize(progress_bar=False)
import subprocess

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### 2. Load test data

In [8]:
def load_10folds_test_data():
    test_path = [f'{cfg.DIR_DATASET}validation/fold{fold_num+1}/valid.feather' for fold_num in range(10)]
    res = [pd.read_feather(path)[['uniprot_id','seq','reaction_id']].rename(columns={'reaction_id': 'rxn_groundtruth'}) for path in test_path]
    return res


print('Loading uniprot_rxn_dict ...' )
d1 = pd.read_feather(cfg.FILE_DS_TRAIN)
d2 = pd.read_feather(cfg.FILE_DS_TEST)
uniprot_rxn_dict = pd.concat([d1,d2], axis=0).reset_index(drop=True)[['uniprot_id', 'reaction_id']].set_index('uniprot_id')['reaction_id'].to_dict()

print('Loading 10-folds test data ...' )
data_test = load_10folds_test_data()
data_test[0].head(3)


Loading uniprot_rxn_dict ...
Loading 10-folds test data ...


,uniprot_id,seq,rxn_groundtruth
105768,Q9UYB6,MLPDRVLEILNEMKAERIRGATWLARKGAEAFLALAEELDEALLED...,-
195319,C1AQW9,MRTPCSQHRRDRPSAIGSQLPDADTLDTRQPPLQEIPISSFADKTF...,RHEA:19669
135884,P64647,MALFSKILIFYVIGVNISFVIIWFISHEKTHIRLLSAFLVGITWPM...,-


### 3. Load Feaatures

In [ ]:
embd_methd = 'tdit5'
file_res_tdit5 = [f'{cfg.RESULTS_DIR}simi/fold_{fold_num}_{embd_methd}_results.h5' for fold_num in range(1,11)]
res_unirep = [btools.read_h5_file(item)for item in tqdm(file_res_unirep)]

In [7]:
tdi_embedding = pd.read_feather(cfg.FILE_DS_3DI_EMBEDDING)

In [12]:
embd_methd = 'unirep'
file_res_unirep = [f'{cfg.RESULTS_DIR}simi/fold_{fold_num}_{embd_methd}_results.h5' for fold_num in range(1,2)]
res_unirep = [btools.read_h5_file(item)for item in tqdm(file_res_unirep)]

100%|██████████| 1/1 [01:00<00:00, 60.06s/it]


In [11]:
file_res_unirep

['/hpcfs/fhome/shizhenkun/codebase/RXNRECer/results/simi/fold_1_unirep_results.h5']

In [16]:
pd.read_feather(cfg.FILE_EMBD_PROTEIN_UNIREP)

,uniprot_id,unirep_file,unirep
0,Q928A4,/hpcfs/fhome/shizhenkun/codebase/preaction/dat...,"[-0.0015948936343193054, 0.003045504679903388,..."
1,Q6G9U3,/hpcfs/fhome/shizhenkun/codebase/preaction/dat...,"[0.007866262458264828, 0.020977269858121872, 0..."
2,Q7VP02,/hpcfs/fhome/shizhenkun/codebase/preaction/dat...,"[0.0046709743328392506, 0.06482962518930435, 0..."
3,Q46WG6,/hpcfs/fhome/shizhenkun/codebase/preaction/dat...,"[0.009201942011713982, -0.039074067026376724, ..."
4,Q8BHJ7,/hpcfs/fhome/shizhenkun/codebase/preaction/dat...,"[0.008545373566448689, -0.020925503224134445, ..."
...,...,...,...
522097,Q731I2,/hpcfs/fhome/shizhenkun/codebase/preaction/dat...,"[0.007942636497318745, 0.23247076570987701, 0...."
522098,P0C798,/hpcfs/fhome/shizhenkun/codebase/preaction/dat...,"[0.009655050002038479, -0.04701140895485878, 0..."
522099,P16609,/hpcfs/fhome/shizhenkun/codebase/preaction/dat...,"[0.008709593676030636, 0.21210989356040955, 0...."
522100,Q49V37,/hpcfs/fhome/shizhenkun/codebase/preaction/dat...,"[0.003473608987405896, -0.03509097173810005, 0..."


### 3. Load results
